In [12]:
!wget -nc https://raw.githubusercontent.com/lazyprogrammer/machine_learning_examples/master/hmm_class/robert_frost.txt

File ‘robert_frost.txt’ already there; not retrieving.



In [13]:
import numpy as np
import string

In [14]:
initial = {} # PI -> for the first word
first_order = {} #A_ij -> for one preceeding word (only)
second_order = {} #A_ijk -> for two or more preceeding words

In [15]:
def remove_punctuation(s):
    return s.translate(str.maketrans('', '', string.punctuation))

#What add2dict() function actually does:


> FOR example: we encounter the lines in the dataset: **'I am happy.'** , **'I am sad.**','**I am content.**', '**I am happy.**','**I am hungry.**','**I am happy.**'

We want to populate the second order dictionary. So, as the key, we need to have the previous two words. The mapping will look like this:
<pre>
{
  ("I","am"):[
              "happy",
              "sad",
              "content",
              "happy",
              "hungry",
              "happy"
  ]
}
</pre>

Which will Later look like this:
<pre>
{ ("I","am"): {
              "happy":0.5,
              "sad":0.07,
              "content":0.35,
              "hungry":0.87 ....
              }
  }
</pre>

In [16]:
def add2dict(dictionary, key, value):
    if key not in dictionary:
        dictionary[key] = []
    dictionary[key].append(value)

In [17]:
for line in open('robert_frost.txt'):
    line = remove_punctuation(line.strip().lower())
    tokens = line.split()
    T = len(tokens)
    for i in range(T):
      t = tokens[i]
      if i == 0:
        # measure the probability of the intial state (first word in the sentence/sequence.)
        initial[t] = initial.get(t, 0.) + 1
      else:
        t_1 = tokens[i-1]
        if i == T-1:
          # measure the probability of ending the line (last word in the sentence/sequence.)
          add2dict(second_order, (t_1, t), 'END')
        if i ==1:
          # measure distribution for second word given first word
          add2dict(first_order, t_1,t)
        else:
          t_2 = tokens[i-2]
          add2dict(second_order, (t_2, t_1 ),t)

In [18]:
#normalize the distribution
initial_sum = sum(initial.values())
for t,key in initial.items():
    initial[t] = key/initial_sum


In [19]:
def list2pdict(ts):
  d = {}
  n = len(ts)
  for t in ts:
    d[t] = d.get(t, 0.) + 1
  for t,c in d.items():
    d[t] = c/n
  return d

In [20]:
for t_1,ts in first_order.items():
  first_order[t_1] = list2pdict(ts)

In [21]:
for k, ts in second_order.items():
  second_order[k] = list2pdict(ts)

In [28]:
def sample_word(d):
  p0 = np.random.random()
  cumulative = 0.
  for t,p in d.items():
    cumulative += p
    if p0 < cumulative:
      return t

In [32]:
def generate():
  for i in range(4):
    sentence = []
    w0 = sample_word(initial)
    sentence.append(w0)
    w1 = sample_word(first_order[w0])
    sentence.append(w1)

    while True:
      w2 = sample_word(second_order[(w0, w1)])
      if w2 == 'END':
        break
      sentence.append(w2)
      w0 = w1
      w1 = w2
    print(' '.join(sentence))

In [34]:
generate()

to tell him everything
of country where two village cultures faded
he broke my trance dont that make you suspicious
and youre her mother
